# 1. Install Dependencies

In [ ]:
# Install required libraries
!pip install -U datasets transformers evaluate peft bitsandbytes
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. Preprocess Data

In [ ]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")
print(imdb)

train_dataset = imdb['train'].shuffle(seed=42)
test_dataset = imdb['test'].shuffle(seed=42)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
from datasets import load_dataset
import torch
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import transformers
import evaluate
from transformers import DataCollatorWithPadding



torch.manual_seed(42)
transformers.set_seed(42)

CHECKPOINT = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='binary')["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average='binary')["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average='binary')["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Set up training arguments with best hyperparameters
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer for final training
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


Step,Training Loss
500,0.293900
1000,0.255800
1500,0.226500
2000,0.135500
2500,0.128400
3000,0.137900
3500,0.080000
4000,0.056200
4500,0.052800


TrainOutput(global_step=4689, training_loss=0.1478620609715438, metrics={'train_runtime': 1433.2779, 'train_samples_per_second': 52.328, 'train_steps_per_second': 3.272, 'total_flos': 1.881768623490624e+16, 'train_loss': 0.1478620609715438, 'epoch': 3.0})


{'eval_loss': 0.27875128388404846, 'eval_accuracy': 0.94108, 'eval_f1': 0.9415267357389544, 'eval_recall': 0.94872, 'eval_precision': 0.9344417303600977, 'eval_runtime': 155.41, 'eval_samples_per_second': 160.865, 'eval_steps_per_second': 10.057, 'epoch': 3.0}


In [ ]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 7435.56 MB


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `NNDL` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when push

In [ ]:
model.push_to_hub("imdb-sft-bert")
tokenizer.push_to_hub("imdb-sft-bert")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yyammerrrss/imdb-sft-bert/commit/62d36d23eb048ad5fb26a6e00b49af10510e7e6b', commit_message='Upload tokenizer', commit_description='', oid='62d36d23eb048ad5fb26a6e00b49af10510e7e6b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yyammerrrss/imdb-sft-bert', endpoint='https://huggingface.co', repo_type='model', repo_id='yyammerrrss/imdb-sft-bert'), pr_revision=None, pr_num=None)

# 4. LoRA

In [ ]:
from peft import LoraConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    modules_to_save=["classifier"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038


In [ ]:

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss
500,0.567200
1000,0.294100
1500,0.277800
2000,0.253300
2500,0.245500
3000,0.243500
3500,0.249300
4000,0.225500
4500,0.236900


TrainOutput(global_step=4689, training_loss=0.2863973933614681, metrics={'train_runtime': 1092.7164, 'train_samples_per_second': 68.636, 'train_steps_per_second': 4.291, 'total_flos': 1.891521648834893e+16, 'train_loss': 0.2863973933614681, 'epoch': 3.0})


{'eval_loss': 0.22236131131649017, 'eval_accuracy': 0.914, 'eval_f1': 0.9146012074992056, 'eval_recall': 0.92104, 'eval_precision': 0.9082518144525087, 'eval_runtime': 165.2821, 'eval_samples_per_second': 151.257, 'eval_steps_per_second': 9.457, 'epoch': 3.0}


In [ ]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 5428.93 MB


# 5. QLoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)

model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear4bit(in_features=768, out_features=768, bias=True)
              (key): Linear4bit(in_features=768, out_features=768, bias=True)
              (value): Linear4bit(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear4bit(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    modules_to_save=["classifier"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 443,906 || all params: 109,927,684 || trainable%: 0.4038


In [ ]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
500,0.607200
1000,0.340000
1500,0.304300
2000,0.271800
2500,0.256700
3000,0.256300
3500,0.265000
4000,0.237300
4500,0.243600


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=4689, training_loss=0.30694291189449585, metrics={'train_runtime': 1594.5603, 'train_samples_per_second': 47.035, 'train_steps_per_second': 2.941, 'total_flos': 1.891521648834893e+16, 'train_loss': 0.30694291189449585, 'epoch': 3.0})


{'eval_loss': 0.2330837845802307, 'eval_accuracy': 0.90936, 'eval_f1': 0.911130284728214, 'eval_recall': 0.92928, 'eval_precision': 0.8936759501461763, 'eval_runtime': 170.5372, 'eval_samples_per_second': 146.596, 'eval_steps_per_second': 9.165, 'epoch': 3.0}


In [ ]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 2082.80 MB


# 6. Prompt Tuning


In [ ]:
from datasets import load_dataset
import torch
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import transformers
import evaluate


torch.manual_seed(42)
transformers.set_seed(42)

CHECKPOINT = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=492)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 110,091,268 || trainable%: 0.5518


In [ ]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 3e-5
best_batch_size = 16
best_epoch = 3
best_weight_decay = 0.1

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)


# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss
500,0.636600
1000,0.614700
1500,0.593900
2000,0.580400
2500,0.573200
3000,0.565700
3500,0.559800
4000,0.564400
4500,0.558800


TrainOutput(global_step=4689, training_loss=0.5822643655072044, metrics={'train_runtime': 1054.0232, 'train_samples_per_second': 71.156, 'train_steps_per_second': 4.449, 'total_flos': 1.894778267192448e+16, 'train_loss': 0.5822643655072044, 'epoch': 3.0})


{'eval_loss': 0.545251727104187, 'eval_accuracy': 0.72588, 'eval_f1': 0.7346575289425795, 'eval_recall': 0.75896, 'eval_precision': 0.7118631349891198, 'eval_runtime': 167.0435, 'eval_samples_per_second': 149.662, 'eval_steps_per_second': 9.357, 'epoch': 3.0}


In [ ]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")


Peak memory usage: 4532.32 MB


It appears that the best hyperparameters we obtained from supervised finetuning does not work well with prompt tuning. This is likely due to the randomly initialised prompt embeddings that require higher learning rate to encode useful information about the underlying data distribution. Following the hyperparameter suggestions by the Prompt Tuning paper authored by Lester et al., we will rerun the training with more training steps, higher learning rate and batch size to see if the evaluation accuracy shows any improvement.  

In [ ]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 110,091,268 || trainable%: 0.5518


In [ ]:
from transformers import EarlyStoppingCallback
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

early_stopping = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)

batch_size = 32
lr = 1e-3
epoch = 15
weight_decay = 0.1

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


Step,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
500,0.585200,0.530461,0.738000,0.693954,0.594080,0.834195
1000,0.520600,0.443825,0.792040,0.811021,0.892480,0.743188
1500,0.454600,0.350326,0.846920,0.845642,0.838640,0.852762
2000,0.395000,0.302844,0.872920,0.873502,0.877520,0.869520
2500,0.366500,0.286843,0.881760,0.879051,0.859360,0.899665
3000,0.340000,0.277789,0.886040,0.887733,0.901120,0.874738
3500,0.329100,0.294781,0.875280,0.882958,0.940880,0.831754
4000,0.312500,0.260645,0.891600,0.893265,0.907200,0.879752
4500,0.312100,0.275666,0.887280,0.892278,0.933680,0.854392
5000,0.307400,0.279990,0.887640,0.892758,0.935360,0.853867


TrainOutput(global_step=5000, training_loss=0.3922983306884766, metrics={'train_runtime': 3717.0976, 'train_samples_per_second': 100.885, 'train_steps_per_second': 3.156, 'total_flos': 4.068466642196352e+16, 'train_loss': 0.3922983306884766, 'epoch': 6.3938618925831205})


{'eval_loss': 0.26064547896385193, 'eval_accuracy': 0.8916, 'eval_f1': 0.893265064986215, 'eval_recall': 0.9072, 'eval_precision': 0.8797517455391777, 'eval_runtime': 158.3502, 'eval_samples_per_second': 157.878, 'eval_steps_per_second': 4.938, 'epoch': 6.3938618925831205}


In [ ]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 8556.42 MB
